Horizontal camera position

In [1]:
import cv2
import numpy as np

# Function to find the X-coordinate of colored objects
def find_x_coordinate(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted_gray_frame = cv2.bitwise_not(gray_frame)
    _, mask = cv2.threshold(inverted_gray_frame, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    x_coordinates = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                cx = int(moments["m10"] / moments["m00"])
                x_coordinates.append(cx)

    return x_coordinates

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    x_coords = find_x_coordinate(frame)

    for cx in x_coords:
        print(f"X-coordinate of the object: {cx}")

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Vertical camera position

In [ ]:
import cv2
import numpy as np

# Function to find the Y-coordinate of colored objects
def find_y_coordinate(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted_gray_frame = cv2.bitwise_not(gray_frame)
    _, mask = cv2.threshold(inverted_gray_frame, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    y_coordinates = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                cy = int(moments["m01"] / moments["m00"])
                y_coordinates.append(cy)

    return y_coordinates

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    y_coords = find_y_coordinate(frame)

    for cy in y_coords:
        print(f"Y-coordinate of the object: {cy}")

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Code that handles both vertical and horizontal camera positions, returning the Y or X coordinates of the detected colored object based on the camera's orientation

In [ ]:
import cv2
import numpy as np

# Function to find the coordinates of colored objects
def find_colored_object_coordinates(frame, lower_color, upper_color, axis='both'):
    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Apply Gaussian Blur to the frame to reduce noise
    blurred_frame = cv2.GaussianBlur(hsv_frame, (11, 11), 0)

    # Create a mask for the specified color
    mask = cv2.inRange(blurred_frame, lower_color, upper_color)

    # Apply morphological operations to remove small noise and close gaps in the mask
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    coordinates = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small contours
            # Get the moments to calculate the centroid
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                cx = int(moments["m10"] / moments["m00"])
                cy = int(moments["m01"] / moments["m00"])

                if axis == 'both':
                    coordinates.append((cx, cy))
                elif axis == 'x':
                    coordinates.append(cx)
                elif axis == 'y':
                    coordinates.append(cy)

                # Draw the contour and centroid on the frame
                cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    return coordinates

# Define the color range for detection (HSV values)
lower_color = np.array([100, 150, 0])  # Lower bound for blue color
upper_color = np.array([140, 255, 255])  # Upper bound for blue color

# Function to process the video and print coordinates
def process_video(axis):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Find the object coordinates
        coords = find_colored_object_coordinates(frame, lower_color, upper_color, axis)

        for coord in coords:
            if axis == 'x':
                print(f"X-coordinate of the object: {coord}")
            elif axis == 'y':
                print(f"Y-coordinate of the object: {coord}")
            else:
                print(f"Coordinates of the object: {coord}")

        # Display the frame
        cv2.imshow("Frame", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()



# For both coordinates:
process_video(axis='both')
